# BACK TRANSLATION

In [ ]:
#Mount drive to access files in gdrive
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
!pip install torchvision 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install transformers[sentencepiece]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.4 MB 12.1 MB/s 
     |████████████████████████████████| 6.6 MB 52.4 MB/s 
     |████████████████████████████████| 596 kB 57.7 MB/s 
     |████████████████████████████████| 101 kB 12.0 MB/s 
     |████████████████████████████████| 1.2 MB 56.9 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
#reading x_train
import pandas as pd

x_train=pd.read_csv('/content/drive/MyDrive/PROJECT/x_train.csv')
x_train.drop(['Unnamed: 0'],axis=1,inplace=True)

In [ ]:
import string
def remove_punctuations(text):
    for punctuation in string.punctuation:
        text = text.replace(punctuation, '')
    return tex

x_train['question'] = x_train['question'].apply(remove_punctuations)
x_train

,im_path,question,answer
0,train2014/COCO_train2014_000000575753.jpg,how many stop signs are in the picture,0
1,train2014/COCO_train2014_000000163599.jpg,what are the boats in,water
2,train2014/COCO_train2014_000000052760.jpg,what color is the sheet the cat is sitting on,white
3,train2014/COCO_train2014_000000573374.jpg,what kind of bear is this,brown
4,train2014/COCO_train2014_000000571773.jpg,what is on the coffee table,candy
...,...,...,...
271771,train2014/COCO_train2014_000000347715.jpg,how many vehicles are parked on the street,5
271772,train2014/COCO_train2014_000000516410.jpg,does the gate have a padlock,no
271773,train2014/COCO_train2014_000000312935.jpg,how many framed pictures are there,2
271774,train2014/COCO_train2014_000000309531.jpg,is the building well lit,yes


In [ ]:
from transformers import MarianMTModel, MarianTokenizer

# Helper function to download data for a language
def download(model_name):
    tokenizer = MarianTokenizer.from_pretrained(model_name)
    model = MarianMTModel.from_pretrained(model_name)
    return tokenizer, model

# download model for English -> French
tmp_lang_tokenizer, tmp_lang_model = download('Helsinki-NLP/opus-mt-en-fr')
# download model for French -> English
src_lang_tokenizer, src_lang_model = download('Helsinki-NLP/opus-mt-fr-en')

Downloading:   0%|          | 0.00/760k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/784k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.28M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.26k [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/760k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.28M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.26k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/287M [00:00<?, ?B/s]

In [ ]:
def translate(texts, model, tokenizer, language):
    """Translate texts into a target language"""
    # Format the text as expected by the model
    formatter_fn = lambda txt: f"{txt}" if language == "en" else f">>{language}<< {txt}"
    original_texts = [formatter_fn(txt) for txt in texts]

    # Translate
    translated = model.generate(**tokenizer(original_texts, return_tensors="pt", padding=True))

    # Decode (tokens to text)
    translated_texts = tokenizer.batch_decode(translated, skip_special_tokens=True)

    return translated_texts

def back_translate(texts, language_src, language_dst):
    """Implements back translation"""
    # Translate from source to target language
    translated = translate(texts, tmp_lang_model, tmp_lang_tokenizer, language_dst)

    # Translate from target language back to source language
    back_translated = translate(translated, src_lang_model, src_lang_tokenizer, language_src)

    return back_translated

In [ ]:
from tqdm import tqdm

def back_translation(x_train):
    da_data = []
    for i in tqdm(range(len(x_train['answer']))):
        answ = x_train['answer'][i]
        im_path = x_train['im_path'][i]
        ques = x_train['question'][i]
        da_data.append({'im_path':im_path,'question':ques,'answer':answ})

        if ((answ != 'yes') and (answ != 'no')):
            aug_ques = back_translate([ques], "en", "fr")
            aug_ques = aug_ques[0].lower()
            if ques != aug_ques:
                da_data.append({'im_path':im_path,'question':aug_ques,'answer':answ})

    x_train_da=pd.DataFrame(da_data)
    return x_train_da

x_train_da = back_translation(x_train)
x_train_da.to_csv('/content/drive/MyDrive/PROJECT/x_train_da.csv')

In [ ]:
y_train = x_train_da['answer']
y_train.to_csv('/content/drive/MyDrive/PROJECT/y_train_da.csv')